In [1]:
import pandas as pd
import json
import ast

In [2]:
df = pd.read_csv("raw/lotofacil.csv")

# lotofacil

In [3]:
# Configurações de diferentes loterias
expected_count = 15

## Limpeza e ajustes nos dados

In [4]:
df = df.reset_index()

In [5]:
## Criar condição aqui
df.drop(columns=['timeCoracao', 'mesSorte', 'trevos'], inplace=True)

In [6]:
df.head()

,index,Unnamed: 0,loteria,concurso,data,local,dezenasOrdemSorteio,dezenas,premiacoes,estadosPremiados,observacao,acumulou,proximoConcurso,dataProximoConcurso,localGanhadores,valorArrecadado,valorAcumuladoConcurso_0_5,valorAcumuladoConcursoEspecial,valorAcumuladoProximoConcurso,valorEstimadoProximoConcurso
0,0,1,lotofacil,656.0,28/07/2011,"Caminhão da Sorte em ROSEIRA, SP","['03', '18', '08', '14', '15', '05', '22', '09...","['02', '03', '05', '07', '08', '09', '11', '13...","[{'descricao': '15 acertos', 'faixa': 1, 'ganh...",[],NaN,0.0,657.0,01/08/2011,"[{'ganhadores': 1, 'municipio': 'MANACAPURU', ...",19064847.50,0.00,0.00,0.0,1700000.0
1,1,6,lotofacil,960.0,25/09/2013,"Caminhão da Sorte em JACOBINA, BA","['16', '14', '01', '06', '07', '02', '24', '17...","['01', '02', '03', '04', '06', '07', '10', '12...","[{'descricao': '15 acertos', 'faixa': 1, 'ganh...",[],NaN,0.0,961.0,27/09/2013,"[{'ganhadores': 1, 'municipio': 'DOURADINA', '...",19443263.75,0.00,3123344.76,0.0,1500000.0
2,2,8,lotofacil,1838.0,10/07/2019,"Espaço Loterias Caixa em SÃO PAULO, SP","['03', '09', '23', '12', '16', '05', '25', '21...","['01', '03', '05', '08', '09', '10', '12', '15...","[{'descricao': '15 acertos', 'faixa': 1, 'ganh...",[],NaN,0.0,1839.0,12/07/2019,"[{'ganhadores': 1, 'municipio': 'SALVADOR', 'n...",25260208.00,0.00,60191753.98,0.0,2000000.0
3,3,9,lotofacil,670.0,15/09/2011,"Caminhão da Sorte em ALEGRETE, RS","['01', '18', '21', '20', '14', '07', '25', '10...","['01', '07', '08', '10', '12', '13', '14', '15...","[{'descricao': '15 acertos', 'faixa': 1, 'ganh...",[],NaN,0.0,671.0,19/09/2011,"[{'ganhadores': 1, 'municipio': 'QUEIMADOS', '...",21563226.25,0.00,0.00,0.0,1800000.0
4,4,11,lotofacil,2348.0,15/10/2021,"ESPAÇO LOTERIAS CAIXA em SÃO PAULO, SP","['15', '24', '22', '13', '19', '08', '01', '05...","['01', '02', '05', '08', '09', '11', '13', '14...","[{'descricao': '15 acertos', 'faixa': 1, 'ganh...",[],NaN,0.0,2349.0,16/10/2021,"[{'ganhadores': 1, 'municipio': 'MARACACUME', ...",19899760.00,2063333.84,11285877.48,0.0,1500000.0


In [7]:
df.columns

Index(['index', 'Unnamed: 0', 'loteria', 'concurso', 'data', 'local',
       'dezenasOrdemSorteio', 'dezenas', 'premiacoes', 'estadosPremiados',
       'observacao', 'acumulou', 'proximoConcurso', 'dataProximoConcurso',
       'localGanhadores', 'valorArrecadado', 'valorAcumuladoConcurso_0_5',
       'valorAcumuladoConcursoEspecial', 'valorAcumuladoProximoConcurso',
       'valorEstimadoProximoConcurso'],
      dtype='object')

### removendo concursos duplicados

In [8]:
if df.duplicated(subset=["concurso"]).sum():
    df = df.drop_duplicates(subset=["concurso"], keep="last")

### corrigindo tipagem de colunas concurso e data (dayfirst por conta do formato brasileiro e mixed pois existem 27/02/2025 e 27-02-2025)

In [9]:
df["concurso"] = pd.to_numeric(df["concurso"], errors="coerce").astype("Int64")

In [10]:
df["data"] = pd.to_datetime(df["data"], dayfirst=True, format='mixed')
df["proximoConcurso"] = pd.to_datetime(df["proximoConcurso"], dayfirst=True, format='mixed')

### corrigindo tipagem de colunas numéricas (dinheiro)

In [11]:
cols_monetarias = [
    "valorArrecadado",
    "valorAcumuladoConcurso_0_5",
    "valorAcumuladoConcursoEspecial",
    "valorAcumuladoProximoConcurso",
    "valorEstimadoProximoConcurso",
]

for c in cols_monetarias:
    df[c] = pd.to_numeric(df[c], errors="coerce")

### corrigindo booleanos

In [12]:
df["acumulou"] = df["acumulou"].astype("boolean")

### normalizar colunas que são listas

In [13]:
def normalizar_lista(lista):
    if isinstance(lista, list):
        return lista
    if isinstance(lista, str):
        try:
            return ast.literal_eval(lista)
        except Exception as e:
            return None
    return None

In [14]:
df["dezenas"] = df["dezenas"].apply(normalizar_lista)
df["dezenasOrdemSorteio"] = df["dezenasOrdemSorteio"].apply(normalizar_lista)

In [15]:
df["premiacoes"] = df["premiacoes"].apply(normalizar_lista)

In [16]:
df["localGanhadores"] = df["localGanhadores"].apply(normalizar_lista)

### valores ausentes
- concursos sem ganhador (localGanhadores)
- arrecadação sem nada (pode ser nulo)
- premiações vazias (válido)

Esses não precisam ser preenchidos

### dezenas inválidas

In [17]:
select_invalido = df["dezenas"].apply(lambda x: isinstance(x, list) and len(x) != expected_count)

In [18]:
select_invalido.sum()

np.int64(0)

In [19]:
df = df[df["dezenas"].apply(lambda x: isinstance(x, list) and len(x) == expected_count)]

### dezenas fora de 1 a 60

In [20]:
def dezenas_validas(lista, loteria):
    intervalos_dezenas = {
    "megasena": (1, 60),
    "lotofacil": (1, 25),
    "timemania": (1, 80),
    "diadesorte": (1, 31),
    "maismilionaria": (1, 50)
    }
    range_dezenas = intervalos_dezenas[loteria]
    if not isinstance(lista, list):
        return False
    return all(range_dezenas[0] <= int(n) <= range_dezenas[1] for n in lista)

In [21]:
df = df[df.apply(lambda row: dezenas_validas(row["dezenas"], row["loteria"]), axis=1)]

### valor monetário negativo

In [22]:
for coluna in cols_monetarias:
    df = df[(df[coluna].isna()) | (df[coluna] >= 0)]

## Engenharia de features

### separando dia, mês, ano, semana do ano e dia da semana das colunas data e proximoConcurso

In [23]:
df['data_dia'] = df['data'].dt.day
df['data_mes'] = df['data'].dt.month
df['data_ano'] = df['data'].dt.year
df['semana_ano_concurso'] = df['data'].dt.isocalendar().week
df["dia_semana_concurso"] = df["data"].dt.weekday

df['proximoConcurso_dia'] = df['proximoConcurso'].dt.day
df['proximoConcurso_mes'] = df['proximoConcurso'].dt.month
df['proximoConcurso_ano'] = df['proximoConcurso'].dt.year

### Separar cada número das dezenas sorteadas em colunas diferentes

In [24]:
df["qtd_dezenas"] = df["dezenas"].apply(lambda x: len(x) if isinstance(x, list) else 0)
max_dezenas = df["qtd_dezenas"].max()

In [25]:
max_dezenas

np.int64(15)

In [26]:
for i in range(max_dezenas):
    df[f"dezena_{i+1}"] = df["dezenas"].apply(
        lambda x: x[i] if isinstance(x, list) and len(x) > i else None
    )

In [27]:
df.drop(columns=["qtd_dezenas"], inplace=True)

### Separar cidade e estado em colunas diferentes (local)

In [28]:
df[["nome_local", "resto"]] = df["local"].str.split(" em ", n=1, expand=True)

In [29]:
df[["cidade", "estado"]] = df["resto"].str.rsplit(", ", n=1, expand=True)

In [30]:
df.drop(columns=["resto"], inplace=True)

### separar premiações em colunas, criar coluna com quantos premiados

In [31]:
df['premiacoes'][1]

[{'descricao': '15 acertos',
  'faixa': 1,
  'ganhadores': 5,
  'valorPremio': 349978.92},
 {'descricao': '14 acertos',
  'faixa': 2,
  'ganhadores': 502,
  'valorPremio': 1532.23},
 {'descricao': '13 acertos',
  'faixa': 3,
  'ganhadores': 20532,
  'valorPremio': 12.5},
 {'descricao': '12 acertos',
  'faixa': 4,
  'ganhadores': 241227,
  'valorPremio': 5},
 {'descricao': '11 acertos',
  'faixa': 5,
  'ganhadores': 1300020,
  'valorPremio': 2.5}]

In [32]:
df["premiacoes"] = df["premiacoes"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str)
    else (x if isinstance(x, list) else None)
)

In [33]:
def expandir_premiacoes(prem_list):
    """
    Recebe a lista de premiações e retorna um dicionário
    com colunas dinâmicas: ganhadores_faixaX e valor_faixaX.
    Funciona para qualquer loteria.
    """
    if not isinstance(prem_list, list):
        return {}

    resultado = {}

    for item in prem_list:
        # identificar faixa
        faixa = item.get("faixa")

        if faixa is None:
            continue

        # criar nomes de coluna dinâmicos
        col_ganhadores = f"ganhadores_faixa_{faixa}"
        col_valor = f"valor_faixa_{faixa}"

        resultado[col_ganhadores] = item.get("ganhadores")
        resultado[col_valor] = item.get("valorPremio")

    return resultado

In [34]:
premios_expandido = df["premiacoes"].apply(expandir_premiacoes)
premios_df = pd.DataFrame(premios_expandido.tolist())
df = pd.concat([df, premios_df], axis=1)

In [35]:
cols_ganhadores = [c for c in df.columns if c.startswith("ganhadores_faixa_")]
df["total_ganhadores"] = df[cols_ganhadores].sum(axis=1)


cols_valores = [c for c in df.columns if c.startswith("valor_faixa_")]
df["total_pago_premios"] = df[cols_valores].sum(axis=1)


df["media_premio_real"] = df["total_pago_premios"] / df["total_ganhadores"]

### localGanhadores separar por coluna

In [36]:
df['localGanhadores'][0]

[{'ganhadores': 1,
  'municipio': 'MANACAPURU',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'AM'},
 {'ganhadores': 1,
  'municipio': 'MANAUS',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'AM'},
 {'ganhadores': 1,
  'municipio': 'SALVADOR',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'BA'},
 {'ganhadores': 1,
  'municipio': 'FORTALEZA',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'CE'},
 {'ganhadores': 1,
  'municipio': 'BRASÍLIA',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'DF'},
 {'ganhadores': 1,
  'municipio': 'ALFENAS',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'MG'},
 {'ganhadores': 1,
  'municipio': 'LAGOA GRANDE',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'MG'},
 {'ganhadores': 1,
  'municipio': 'PINHAIS',
  'nomeFatansiaUL': '',
  'serie': '',
  'posicao': 1,
  'uf': 'PR'},
 {'ganhadores': 1,
  'municipio': 'SÃO GONÇALO',
  'nomeFatansiaUL': 

In [37]:
def normalizar_local(lista):
    if isinstance(lista, list):
        return lista
    return []

df["localGanhadores"] = df["localGanhadores"].apply(normalizar_local)

In [38]:
df["municipioGanhador"] = df["localGanhadores"].apply(
    lambda x: x[0].get("municipio") if len(x) > 0 else None
)

df["ufGanhador"] = df["localGanhadores"].apply(
    lambda x: x[0].get("uf") if len(x) > 0 else None
)

In [39]:
def is_ticket_online(x):
    if len(x) == 0:
        return False
    registro = x[0]
    municipio = registro.get("municipio", "").strip().upper()
    uf = registro.get("uf", "").strip().upper()

    return municipio == "CANAL ELETRONICO" or uf == "BR"

df["ticketGanhadorOnline"] = df["localGanhadores"].apply(is_ticket_online)

In [40]:
df['ticketGanhadorOnline'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2815 entries, 0 to 2814
Series name: ticketGanhadorOnline
Non-Null Count  Dtype
--------------  -----
2815 non-null   bool 
dtypes: bool(1)
memory usage: 2.9 KB


In [41]:
df['ufGanhador'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2815 entries, 0 to 2814
Series name: ufGanhador
Non-Null Count  Dtype 
--------------  ----- 
2491 non-null   object
dtypes: object(1)
memory usage: 22.1+ KB


### verificar diferença entre nulo e 0 em valorArrecadado

In [42]:
df['valorArrecadado']

0       19064847.50
1       19443263.75
2       25260208.00
3       21563226.25
4       19899760.00
           ...     
2810    48113632.50
2811    18383551.25
2812    32148292.50
2813    15449486.25
2814    26683920.00
Name: valorArrecadado, Length: 2815, dtype: float64

In [43]:
df["valorArrecadado"] = df["valorArrecadado"].replace(0, None)

### criar coluna de razão entre valor estimado e valor acumulado

In [44]:
def calcular_razao(row):
    acumulado = row["valorAcumuladoProximoConcurso"]
    estimado  = row["valorEstimadoProximoConcurso"]

    # evita problemas de None, NaN ou divisões por zero
    if acumulado is None or pd.isna(acumulado) or acumulado == 0:
        return None
    if estimado is None or pd.isna(estimado):
        return None

    return estimado / acumulado

df["razaoEstimadoAcumulado"] = df.apply(calcular_razao, axis=1)

### colunas extras

In [45]:
def normalizar_dezenas(lista):
    if not isinstance(lista, list):
        return None
    nova = []
    for item in lista:
        try:
            nova.append(int(item))  # converte "03" → 3
        except:
            return None  # caso algo esteja muito errado
    return nova

df["dezenas"] = df["dezenas"].apply(normalizar_dezenas)

In [46]:
df["qtd_pares"] = df["dezenas"].apply(
    lambda x: sum(1 for n in x if n % 2 == 0) if isinstance(x, list) else None
)

df["qtd_impares"] = df["dezenas"].apply(
    lambda x: sum(1 for n in x if n % 2 != 0) if isinstance(x, list) else None
)


df["range_dezenas"] = df["dezenas"].apply(
    lambda x: max(x) - min(x) if isinstance(x, list) and len(x) == 6 else None
)